In [1]:
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import torch
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import time

### Load data

In [2]:
!wget --random-wait -i ../data/georges.csv -P ../data/images/georges
!wget --random-wait -i ../data/non_georges.csv -P ../data/images/non_georges

--2021-02-23 00:13:38--  https://i.pinimg.com/736x/17/0d/5b/170d5b93d80d247be60f22ca1216bef7.jpg
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving i.pinimg.com (i.pinimg.com)... 2.18.76.196, 2001:2030:21:180::1931, 2001:2030:21:18d::1931, ...
Connecting to i.pinimg.com (i.pinimg.com)|2.18.76.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155241 (152K) [image/jpeg]
Saving to: ‘../data/images/georges/170d5b93d80d247be60f22ca1216bef7.jpg’

170d5b93d80d247be60 100%[===================>] 151.60K  --.-KB/s    in 0.07s   

2021-02-23 00:13:39 (1.98 MB/s) - ‘../data/images/georges/170d5b93d80d247be60f22ca1216bef7.jpg’ saved [155241/155241]

--2021-02-23 00:13:39--  https://i.pinimg.com/736x/47/b9/9a/47b99a2ddcde2cf9e62b4faeec2fdf20.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 62029 (61K) [image/jpeg]
Saving to: ‘../data/images/georges/47b99a2ddcde2cf9e62b4faeec2fdf20.jpg’

47b99a2dd

HTTP request sent, awaiting response... 200 OK
Length: 30151 (29K) [image/jpeg]
Saving to: ‘../data/images/georges/e4b920e0ea415702143e3ba1427ac31e.jpg’

e4b920e0ea415702143 100%[===================>]  29.44K  --.-KB/s    in 0.008s  

2021-02-23 00:13:43 (3.82 MB/s) - ‘../data/images/georges/e4b920e0ea415702143e3ba1427ac31e.jpg’ saved [30151/30151]

--2021-02-23 00:13:43--  https://i.pinimg.com/736x/93/e3/5c/93e35c89b4ad8135e9453f49f6e26480.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 138415 (135K) [image/jpeg]
Saving to: ‘../data/images/georges/93e35c89b4ad8135e9453f49f6e26480.jpg’

93e35c89b4ad8135e94 100%[===================>] 135.17K  --.-KB/s    in 0.03s   

2021-02-23 00:13:43 (4.74 MB/s) - ‘../data/images/georges/93e35c89b4ad8135e9453f49f6e26480.jpg’ saved [138415/138415]

--2021-02-23 00:13:43--  https://i.pinimg.com/736x/f3/dd/32/f3dd3201859ff67b986c4fe681788db5.jpg
Reusing existing connection to i.pinimg.com:443.


HTTP request sent, awaiting response... 200 OK
Length: 140669 (137K) [image/jpeg]
Saving to: ‘../data/images/georges/4e8b08087491ea7935b6cf5ddc76f384.jpg’

4e8b08087491ea7935b 100%[===================>] 137.37K  --.-KB/s    in 0.03s   

2021-02-23 00:13:51 (4.30 MB/s) - ‘../data/images/georges/4e8b08087491ea7935b6cf5ddc76f384.jpg’ saved [140669/140669]

--2021-02-23 00:13:51--  https://i.pinimg.com/736x/4b/a6/3b/4ba63b103db5030ba664787347b0c64c.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 124496 (122K) [image/jpeg]
Saving to: ‘../data/images/georges/4ba63b103db5030ba664787347b0c64c.jpg’

4ba63b103db5030ba66 100%[===================>] 121.58K  --.-KB/s    in 0.03s   

2021-02-23 00:13:51 (4.20 MB/s) - ‘../data/images/georges/4ba63b103db5030ba664787347b0c64c.jpg’ saved [124496/124496]

--2021-02-23 00:13:51--  https://i.pinimg.com/736x/37/2f/08/372f08758503e7be89e09617ea9bea35.jpg
Reusing existing connection to i.pinimg.com:4

HTTP request sent, awaiting response... 200 OK
Length: 100958 (99K) [image/jpeg]
Saving to: ‘../data/images/georges/03a3e96b15e8735b097156d7b138cd92.jpg’

03a3e96b15e8735b097 100%[===================>]  98.59K   371KB/s    in 0.3s    

2021-02-23 00:13:55 (371 KB/s) - ‘../data/images/georges/03a3e96b15e8735b097156d7b138cd92.jpg’ saved [100958/100958]

--2021-02-23 00:13:55--  https://i.pinimg.com/736x/92/f0/4e/92f04ea993da153b5c7c88ac48814510.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 196723 (192K) [image/jpeg]
Saving to: ‘../data/images/georges/92f04ea993da153b5c7c88ac48814510.jpg’

92f04ea993da153b5c7 100%[===================>] 192.11K  --.-KB/s    in 0.05s   

2021-02-23 00:13:55 (3.72 MB/s) - ‘../data/images/georges/92f04ea993da153b5c7c88ac48814510.jpg’ saved [196723/196723]

--2021-02-23 00:13:55--  https://i.pinimg.com/736x/26/eb/5a/26eb5abeff7d7f515d0096fb960aa471.jpg
Reusing existing connection to i.pinimg.com:443

HTTP request sent, awaiting response... 200 OK
Length: 45324 (44K) [image/jpeg]
Saving to: ‘../data/images/georges/7e3096fa9a3ce2496aa046bc7df61d50.jpg’

7e3096fa9a3ce2496aa 100%[===================>]  44.26K  --.-KB/s    in 0.008s  

2021-02-23 00:14:02 (5.35 MB/s) - ‘../data/images/georges/7e3096fa9a3ce2496aa046bc7df61d50.jpg’ saved [45324/45324]

--2021-02-23 00:14:02--  https://i.pinimg.com/736x/f0/28/60/f02860084f298aca9469f809d8afb897.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 237289 (232K) [image/jpeg]
Saving to: ‘../data/images/georges/f02860084f298aca9469f809d8afb897.jpg’

f02860084f298aca946 100%[===================>] 231.73K  --.-KB/s    in 0.06s   

2021-02-23 00:14:02 (3.98 MB/s) - ‘../data/images/georges/f02860084f298aca9469f809d8afb897.jpg’ saved [237289/237289]

--2021-02-23 00:14:02--  https://i.pinimg.com/736x/15/33/11/153311a3697f0602feb46968e5323488.jpg
Reusing existing connection to i.pinimg.com:443.


...

HTTP request sent, awaiting response... 200 OK
Length: 113416 (111K) [image/jpeg]
Saving to: ‘../data/images/non_georges/165f57630d1f8afdb1284ee1007b34ff.jpg’

165f57630d1f8afdb12 100%[===================>] 110.76K  --.-KB/s    in 0.03s   

2021-02-23 00:26:58 (3.68 MB/s) - ‘../data/images/non_georges/165f57630d1f8afdb1284ee1007b34ff.jpg’ saved [113416/113416]

--2021-02-23 00:26:58--  https://i.pinimg.com/736x/8b/5c/b1/8b5cb1d4901f8585eebfa1c47cd88f09.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 80676 (79K) [image/jpeg]
Saving to: ‘../data/images/non_georges/8b5cb1d4901f8585eebfa1c47cd88f09.jpg’

8b5cb1d4901f8585eeb 100%[===================>]  78.79K  --.-KB/s    in 0.01s   

2021-02-23 00:26:58 (5.38 MB/s) - ‘../data/images/non_georges/8b5cb1d4901f8585eebfa1c47cd88f09.jpg’ saved [80676/80676]

--2021-02-23 00:26:58--  https://i.pinimg.com/736x/81/73/4d/81734dd8b605bc8e4fc6c72fe28271b1.jpg
Reusing existing connection to i.

HTTP request sent, awaiting response... 200 OK
Length: 53320 (52K) [image/jpeg]
Saving to: ‘../data/images/non_georges/9f9d4e5944bdd069a0ea077c14527f83.jpg’

9f9d4e5944bdd069a0e 100%[===================>]  52.07K  --.-KB/s    in 0.01s   

2021-02-23 00:26:59 (5.24 MB/s) - ‘../data/images/non_georges/9f9d4e5944bdd069a0ea077c14527f83.jpg’ saved [53320/53320]

--2021-02-23 00:26:59--  https://i.pinimg.com/736x/90/73/04/907304074f4744df9f260aaa5bfb485f.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 68216 (67K) [image/jpeg]
Saving to: ‘../data/images/non_georges/907304074f4744df9f260aaa5bfb485f.jpg’

907304074f4744df9f2 100%[===================>]  66.62K  --.-KB/s    in 0.01s   

2021-02-23 00:26:59 (5.60 MB/s) - ‘../data/images/non_georges/907304074f4744df9f260aaa5bfb485f.jpg’ saved [68216/68216]

--2021-02-23 00:26:59--  https://i.pinimg.com/736x/a1/c2/77/a1c27787cd08d5c6c922f9b42d27472d.jpg
Reusing existing connection to i.pini

HTTP request sent, awaiting response... 200 OK
Length: 23066 (23K) [image/jpeg]
Saving to: ‘../data/images/non_georges/c06736b89940a1b2fddc02b46d715e1b.jpg’

c06736b89940a1b2fdd 100%[===================>]  22.53K  --.-KB/s    in 0.003s  

2021-02-23 00:27:00 (6.79 MB/s) - ‘../data/images/non_georges/c06736b89940a1b2fddc02b46d715e1b.jpg’ saved [23066/23066]

--2021-02-23 00:27:00--  https://i.pinimg.com/736x/71/8f/d4/718fd469f31654294146912e6f1ce615.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 103281 (101K) [image/jpeg]
Saving to: ‘../data/images/non_georges/718fd469f31654294146912e6f1ce615.jpg’

718fd469f3165429414 100%[===================>] 100.86K  --.-KB/s    in 0.03s   

2021-02-23 00:27:00 (3.06 MB/s) - ‘../data/images/non_georges/718fd469f31654294146912e6f1ce615.jpg’ saved [103281/103281]

--2021-02-23 00:27:00--  https://i.pinimg.com/736x/15/49/a0/1549a0a1ca77aaa562aca58f299ac5dd.jpg
Reusing existing connection to i.

HTTP request sent, awaiting response... 200 OK
Length: 170189 (166K) [image/jpeg]
Saving to: ‘../data/images/non_georges/4f47f830513d25dbc3ed149fe650ad35.jpg’

4f47f830513d25dbc3e 100%[===================>] 166.20K  --.-KB/s    in 0.04s   

2021-02-23 00:27:01 (3.79 MB/s) - ‘../data/images/non_georges/4f47f830513d25dbc3ed149fe650ad35.jpg’ saved [170189/170189]

--2021-02-23 00:27:01--  https://i.pinimg.com/736x/9b/18/88/9b1888fd64aa7a90b3030982b7bb169b.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 92714 (91K) [image/jpeg]
Saving to: ‘../data/images/non_georges/9b1888fd64aa7a90b3030982b7bb169b.jpg’

9b1888fd64aa7a90b30 100%[===================>]  90.54K  --.-KB/s    in 0.01s   

2021-02-23 00:27:01 (6.00 MB/s) - ‘../data/images/non_georges/9b1888fd64aa7a90b3030982b7bb169b.jpg’ saved [92714/92714]

--2021-02-23 00:27:01--  https://i.pinimg.com/736x/85/49/1a/85491a7426a95f1de547504038094333.jpg
Reusing existing connection to i.

HTTP request sent, awaiting response... 200 OK
Length: 296700 (290K) [image/jpeg]
Saving to: ‘../data/images/non_georges/776753c14eeb066cbb0e02114d40219a.jpg’

776753c14eeb066cbb0 100%[===================>] 289.75K  --.-KB/s    in 0.06s   

2021-02-23 00:27:02 (4.46 MB/s) - ‘../data/images/non_georges/776753c14eeb066cbb0e02114d40219a.jpg’ saved [296700/296700]

--2021-02-23 00:27:02--  https://i.pinimg.com/736x/e5/d0/f4/e5d0f428313ae6d8b98626dad1d9965e.jpg
Reusing existing connection to i.pinimg.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 36514 (36K) [image/jpeg]
Saving to: ‘../data/images/non_georges/e5d0f428313ae6d8b98626dad1d9965e.jpg’

e5d0f428313ae6d8b98 100%[===================>]  35.66K  --.-KB/s    in 0.009s  

2021-02-23 00:27:02 (4.03 MB/s) - ‘../data/images/non_georges/e5d0f428313ae6d8b98626dad1d9965e.jpg’ saved [36514/36514]

--2021-02-23 00:27:02--  https://i.pinimg.com/736x/ba/d6/2b/bad62bc6efa3883413df693fb6b48a87.jpg
Reusing existing connection to i.

#### Remove dublicates

In [3]:
!rm ../data/images/*/*.1
!rm ../data/images/*/*.2
!rm ../data/images/*/*.3

#### Visualize a few images

In [ ]:
fig = plt.figure(figsize=(16, 17))
axes = []
path = '../data/images/georges'
for i, img_name in zip(np.arange(1, 21), os.listdir(path)[:20]):
    axes.append(fig.add_subplot(4, 5, i))
    subplot_title = ("Image" + str(i))
    axes[-1].set_title(subplot_title) 
    img_path = os.path.join(path, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)

In [ ]:
fig = plt.figure(figsize=(16, 17))
axes = []
path = '../data/images/non_georges'
for i, img_name in zip(np.arange(1, 21), os.listdir(path)[:20]):
    axes.append(fig.add_subplot(4, 5, i))
    subplot_title = ("Image" + str(i))
    axes[-1].set_title(subplot_title) 
    img_path = os.path.join(path, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)

### Define parameters

In [6]:
train_size = 0.7
feature_extract = True
num_classes = 2
input_size = 224 # Resnet50
num_workers = 4
batch_size = 32
lr = 0.001
momentum = 0.9
criterion = nn.CrossEntropyLoss()
num_epochs = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
np.random.seed(0)
torch.manual_seed(0)
torch.set_deterministic(True)

### Define and initialize model

In [8]:
model = models.resnet50(pretrained=True)

if feature_extract:
    for param in model.parameters():
        param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

### Setup transforms, datasets, and dataloaders

In [9]:
dataset = datasets.ImageFolder('../data/images')

num_imgs = len(dataset)
indices = list(range(num_imgs))
class_names = dataset.classes

split = int(train_size * num_imgs)
np.random.shuffle(indices)
train_idx, val_idx= indices[:split], indices[split:]

train_data = Subset(dataset, indices=train_idx)
val_data = Subset(dataset, indices=val_idx)

train_data.dataset = copy.deepcopy(dataset)
val_data.dataset = copy.deepcopy(dataset)

train_trans = transforms.Compose([
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_trans = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_data.dataset.transform = train_trans
val_data.dataset.transform = val_trans

dataloaders_dict = {'train' : DataLoader(train_data, batch_size=batch_size,
                                         num_workers=num_workers, shuffle=True),
                    'val' : DataLoader(val_data, batch_size=batch_size, 
                                       num_workers=num_workers, shuffle=True)
                   }

### Create the optimizer

In [10]:
params_to_update = model.parameters()
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)

optimizer = optim.SGD(params_to_update, lr=lr, momentum=momentum)

### Train and evaluate

In [11]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [12]:
def visualize_model(model, num_images=6):
    mean = np.array([0.485, 0.456, 0.406])
    std =  np.array([0.229, 0.224, 0.225])
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure(figsize=(16, 17))

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders_dict['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images // 2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}, true label: {class_names[labels[j]]}')
                img = inputs.cpu().data[j].numpy().transpose((1, 2, 0)) * std + mean
                img = np.clip(img, 0, 1)
                plt.imshow(img)

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [13]:
model, hist = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/5
----------
train Loss: 0.5517 Acc: 0.7207
val Loss: 0.4170 Acc: 0.8141

Epoch 2/5
----------
train Loss: 0.4706 Acc: 0.7681
val Loss: 0.3887 Acc: 0.8270

Epoch 3/5
----------
train Loss: 0.4300 Acc: 0.8007
val Loss: 0.3704 Acc: 0.8399

Epoch 4/5
----------
train Loss: 0.4484 Acc: 0.7947
val Loss: 0.3923 Acc: 0.8171

Epoch 5/5
----------
train Loss: 0.4303 Acc: 0.8030
val Loss: 0.4678 Acc: 0.7797

Training complete in 111m 1s
Best val Acc: 0.839860


In [ ]:
visualize_model(model, 12)

In [15]:
torch.save(model.state_dict(), '../models/resnet50-feature-extraction.pt')